In [13]:
import optuna
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import tqdm
from sklearn.metrics import roc_curve, auc

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
dimensionality = [9, 12, 15, 18]

In [ ]:
class DynamicNeuralNetwork(torch.nn.Module):
    def __init__(self, dim, hidden_sizes, dropout_rate):
        super(DynamicNeuralNetwork, self).__init__()
        layers = []
        last_size = dim
        for size in hidden_sizes:
            layers.append(torch.nn.Linear(last_size, size))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(dropout_rate))
            last_size = size
        layers.append(torch.nn.Linear(last_size, 1))
        layers.append(torch.nn.Sigmoid())
        self.network = torch.nn.Sequential(*layers)

        # Initialize weights, based on https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/Basics/pytorch_init_weights.py
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                torch.nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        return self.network(x)

In [ ]:
def denoise_bernoulli_data(X):
    """Denoise data by converting to binary values"""
    return (X >= 0.5).astype(float)

In [ ]:
def create_dataloaders(X, y, batch_size):
    """Convert numpy arrays to PyTorch tensors and create DataLoaders."""
    X = denoise_bernoulli_data(X)
    X = torch.tensor(X, dtype=torch.float32).to(device)
    y = torch.tensor(y, dtype=torch.long).to(device)

    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    return loader

In [ ]:
def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device, early_stopping_patience=50, print_statements=False):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    patience_counter = 0
    
    best_val_loss = float('inf')
    for epoch in tqdm.tqdm(range(num_epochs)):
        # Train Model
        model.to(device)
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Calculate accuracy
            predicted = (outputs >= 0.5).float()
            train_correct += (predicted == labels.unsqueeze(1)).sum().item()
            train_total += labels.size(0)
        
        train_loss = train_loss / len(train_loader)
        train_accuracy = train_correct / train_total * 100
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Evaluate the model
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels.unsqueeze(1).float())
                
                val_loss += loss.item()

                # Calculate accuracy
                predicted = (outputs >= 0.5).float()
                val_correct += (predicted == labels.unsqueeze(1)).sum().item()
                val_total += labels.size(0)

        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1


        if patience_counter >= early_stopping_patience and print_statements:
            print(f'Early stopping at epoch {epoch + 1}')
            break
        
        if (epoch + 1) % 10 == 0 and print_statements:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    return model, best_val_loss, train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
def evaluate_model(model, dim, test_loader, save_file):
	model.eval()
	all_preds = []
	all_labels = []
    
	with torch.no_grad():
		for inputs, labels in test_loader:
			# Ensure inputs are in the correct shape
			inputs = inputs.view(inputs.size(0), -1)  # Flatten if necessary
			outputs = model(inputs)
			predictions = (outputs >= 0.5).float().cpu().numpy()
			all_preds.extend(predictions)
			all_labels.extend(labels.cpu().numpy())
    
	all_preds = np.array(all_preds).squeeze()
	print(all_preds)
	
	if save_file:    
		all_labels = np.array(all_labels)
		np.save(f'kryptonite_{dim}_pred_nn.npy', all_preds)

	return {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds),
        'recall': recall_score(all_labels, all_preds),
        'f1': f1_score(all_labels, all_preds)
    }

In [ ]:
def get_predictions(model, test_loader):
	model.eval()
	all_preds = []
	all_labels = []
    
	with torch.no_grad():
		for inputs, labels in test_loader:
			# Ensure inputs are in the correct shape
			inputs = inputs.view(inputs.size(0), -1)  # Flatten if necessary
			outputs = model(inputs).float().cpu().numpy()
			predictions = (outputs >= 0.5).float().cpu().numpy()
			all_preds.extend(predictions)
			all_labels.extend(labels.cpu().numpy())

	all_preds = np.array(all_preds).squeeze()
	print(all_preds)

	return {
		'all_predictions': all_preds,
		'all_labels': all_labels
    }

In [ ]:
def get_predictions_final(model, test_loader):
	model.eval()
	all_preds = []

	with torch.no_grad():
		for inputs in test_loader:
			# Ensure inputs are in the correct shape
			inputs = inputs.view(inputs.size(0), -1)  # Flatten if necessary
			outputs = model(inputs).float().cpu().numpy()
			predictions = (outputs >= 0.5).float().cpu().numpy()
			all_preds.extend(predictions)

	all_preds = np.array(all_preds).squeeze()
	print(all_preds)

	return {
		'all_predictions': all_preds
    }

In [ ]:
def get_predictions_prob(model, test_loader):
	model.eval()
	all_preds = []
	all_labels = []

	with torch.no_grad():
		for inputs, labels in test_loader:
			# Ensure inputs are in the correct shape
			inputs = inputs.view(inputs.size(0), -1)  # Flatten if necessary
			outputs = model(inputs).float().cpu().numpy()
			all_preds.extend(outputs)
			all_labels.extend(labels.cpu().numpy())
    
	all_preds = np.array(all_preds).squeeze()
	print(all_preds)

	return {
		'all_predictions': all_preds,
		'all_labels': all_labels
    }

In [ ]:
def k_fold_cross_validation(X, y, best_params, dim, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    results = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train_loader = create_dataloaders(X_train, y_train, best_params['batch_size'])
        val_loader = create_dataloaders(X_test, y_test, best_params['batch_size'])

        model = DynamicNeuralNetwork(dim, best_params['hidden_sizes'], best_params['dropout_rate']).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
        loss_fn = torch.nn.BCELoss()

        trained_model, _, _, _, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, 100, device)
        results.append(evaluate_model(trained_model, dim, val_loader, False))

    # Calculate metrics across all folds
    metric_names = ['accuracy', 'precision', 'recall', 'f1']
    avg_metrics = {
        metric: np.mean([result[metric] for result in results])
        for metric in metric_names
    }
    
    std_metrics = {
        metric: np.std([result[metric] for result in results])
        for metric in metric_names
    }
    
    # Print summary statistics
    print("\nAverage Results Across All Folds:")
    for metric in metric_names:
        print(f"{metric.capitalize()}: {avg_metrics[metric]:.4f} ± {std_metrics[metric]:.4f}")
    
    return avg_metrics, std_metrics

In [ ]:
def objective(trial, dim):
	X_train = np.load(f'Datasets_Train_Test_Split/kryptonite-{dim}-X_train.npy')
	y_train = np.load(f'Datasets_Train_Test_Split/kryptonite-{dim}-y_train.npy')

	X_train = denoise_bernoulli_data(X_train)

	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

	batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
	learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.01, step=0.001)
	weight_decay = 0.0001
	dropout_rate = 0.15
	num_layers = trial.suggest_int('num_layers', 2, 4)
	hidden_sizes = [trial.suggest_categorical(f'hidden_size_l{i}', [64, 128, 256]) for i in range(num_layers)]
	num_epochs = 200

	train_loader = create_dataloaders(X_train, y_train, batch_size)
	val_loader = create_dataloaders(X_val, y_val, batch_size)
	model = DynamicNeuralNetwork(dim, hidden_sizes=hidden_sizes, dropout_rate=dropout_rate).to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	loss_fn = torch.nn.BCELoss()
	_, _, _, val_losses, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs, device)

	return val_losses[-1]

In [ ]:
best_params_by_dim = []

for dim in dimensionality:
	print(f"-=-=-=-= Running Dimension: {dim} =-=-=-=-")
	study = optuna.create_study(direction='minimize')
	study.optimize(lambda trial: objective(trial, dim), n_trials=10)

	print("Best trial:")
	trial = study.best_trial

	print(f"  Value: {trial.value}")
	print("  Params: ")
	for key, value in trial.params.items():
		print(f"    {key}: {value}")

	best_params = trial.params
	best_params['hidden_sizes'] = [best_params.pop(f'hidden_size_l{i}') for i in range(best_params['num_layers'])]
	del best_params['num_layers']
	best_params_by_dim.append(best_params)

In [ ]:
def plot_roc_curve(y_true, y_pred_proba):
    """
    Plot ROC curve and calculate AUC score from predictions and true values.
    
    Parameters:
    -----------
    y_true : array-like
        True binary labels (0, 1)
    y_pred_proba : array-like
        Predicted probabilities for the positive class
        
    Returns:
    --------
    float
        AUC score
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.fill_between(fpr, tpr, alpha=0.2, color='darkorange')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    
    return roc_auc

In [ ]:
def plot_multi_roc_curves(y_true_list, y_pred_proba_list, dimensions):
    """
    Plot ROC curves for multiple dimensions on the same figure.
    
    Parameters:
    -----------
    y_true_list : list of array-like
        List of true binary labels for each dimension
    y_pred_proba_list : list of array-like
        List of predicted probabilities for each dimension
    dimensions : list
        List of dimension values
    
    Returns:
    --------
    dict
        Dictionary of AUC scores for each dimension
    """
    plt.figure(figsize=(10, 8))
    auc_scores = {}
    
    colors = plt.cm.rainbow(np.linspace(0, 1, len(dimensions)))
    
    for (y_true, y_pred_proba, dim, color) in zip(y_true_list, y_pred_proba_list, dimensions, colors):
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        auc_scores[dim] = roc_auc
        
        plt.plot(fpr, tpr, color=color, lw=2, 
                label=f'Dim {dim} (AUC = {roc_auc:.2f})')
        plt.fill_between(fpr, tpr, alpha=0.1, color=color)
    
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=22)
    plt.ylabel('True Positive Rate', fontsize=22)
    plt.title('ROC Curves for Different Dimensions', fontsize=22)
    plt.legend(loc="lower right", bbox_to_anchor=(1.15, 0), fontsize=22)
    plt.grid(True, alpha=0.3)
    
    return auc_scores

In [ ]:
y_true_list = []
y_pred_proba_list = []

for i, dim in enumerate(dimensionality):
	print(f"-=-=-=-= Running {dim} =-=-=-=-")
	X_train = np.load(r'Datasets_Train_Test_Split/kryptonite-{dim}-X_train.npy')
	X_test = np.load(r'Datasets_Train_Test_Split/kryptonite-{dim}-X_test.npy')
	y_train = np.load(r'Datasets_Train_Test_Split/kryptonite-{dim}-y_train.npy')
	y_test = np.load(r'Datasets_Train_Test_Split/kryptonite-{dim}-y_test.npy')

	X_train = denoise_bernoulli_data(X_train)
	X_test = denoise_bernoulli_data(X_test)

	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
	best_params = best_params_by_dim[i]
	train_loader = create_dataloaders(X_train, y_train, best_params['batch_size'])
	val_loader = create_dataloaders(X_val, y_val, best_params['batch_size'])
	test_loader = create_dataloaders(X_test, y_test, best_params['batch_size'])
	model = DynamicNeuralNetwork(dim, best_params['hidden_sizes'], 0.15).to(device)
	optimizer = torch.optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=0.0001)
	loss_fn = torch.nn.BCELoss()
	trained_model, _, _, _, _, _ = train_model(model, train_loader, val_loader, loss_fn, optimizer, 200, device)
	test_metrics_run = evaluate_model(trained_model, dim, test_loader, False)
	for metric, value in test_metrics_run.items():
		print(f"{metric.capitalize()}: {value:.4f}")

	predictions = get_predictions_prob(trained_model, test_loader)
	y_true_list.append(predictions['all_labels'])
	y_pred_proba_list.append(predictions['all_predictions'])

	plot_roc_curve(predictions['all_labels'], predictions['all_predictions'])

In [ ]:
plot_multi_roc_curves(y_true_list, y_pred_proba_list, dimensionality)

In [ ]:
for i, dim in enumerate(dimensionality):
	print(f"-=-=-=-= Running {dim} =-=-=-=-")

	best_params = best_params_by_dim[i]

	X = np.load(f'Datasets_Train_Test_Split/kryptonite-{dim}-X.npy')
	y = np.load(f'Datasets_Train_Test_Split/kryptonite-{dim}-y.npy')

	X = denoise_bernoulli_data(X)

	# Get both metrics and standard deviations
	avg_metrics, std_metrics = k_fold_cross_validation(X, y, best_params, dim, k=2)

	# Print results with proper formatting and ± symbol
	print("\nFinal K-Fold Cross Validation Results:")
	print(f"Average Accuracy:  {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
	print(f"Average Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
	print(f"Average Recall:    {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
	print(f"Average F1:        {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")